# Neural Machine Translation

**Reference:** Sutskever, Ilya, Oriol Vinyals, and Quoc V. Le. "Sequence to sequence learning with neural networks." In Advances in neural information processing systems, pp. 3104-3112. 2014. ([Paper](https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks), [Sample code](https://github.com/tensorflow/nmt))

In [1]:
using Pkg
packages = ["Knet", "Test", "IterTools", "Random"]
for p in packages; Pkg.add(p); end
Pkg.update("Knet")
using Knet, Test, Base.Iterators, IterTools, Random # , LinearAlgebra, StatsBase
using AutoGrad: @gcheck  # to check gradients, usew with Float64
Knet.atype() = KnetArray{Float32}  # determines what Knet.param() uses.
macro size(z, s); esc(:(@assert (size($z) == $s) string(summary($z),!=,$s))); end # for debugging

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no cha

@size (macro with 1 method)

## Part -1. Types from the last project

Please copy the following types and related functions from the last project: `Vocab`,
`TextReader`, `Embed`, `Linear`, `mask!`.

In [2]:
# Your code here
include("hw2_functions.jl");

## Part 0. Load data

We will use the Turkish-English pair from the [TED Talks Dataset](https://github.com/neulab/word-embeddings-for-nmt) for our experiments.

In [3]:
datadir = "datasets/tr_to_en"

if !isdir(datadir);
    download("http://www.phontron.com/data/qi18naacl-dataset.tar.gz", "qi18naacl-dataset.tar.gz")
    run(`tar xzf qi18naacl-dataset.tar.gz`)
end

if !isdefined(Main, :tr_vocab)
    tr_vocab = Vocab("$datadir/tr.train", mincount=5)
    en_vocab = Vocab("$datadir/en.train", mincount=5)
    tr_train = TextReader("$datadir/tr.train", tr_vocab)
    en_train = TextReader("$datadir/en.train", en_vocab)
    tr_dev = TextReader("$datadir/tr.dev", tr_vocab)
    en_dev = TextReader("$datadir/en.dev", en_vocab)
    tr_test = TextReader("$datadir/tr.test", tr_vocab)
    en_test = TextReader("$datadir/en.test", en_vocab)
    @info "Testing data"
    @test length(tr_vocab.i2w) == 38126
    @test length(first(tr_test)) == 16
    @test length(collect(tr_test)) == 5029
end

┌ Info: Testing data
└ @ Main In[3]:17


Test Passed

## Part 1. Minibatching

For minibatching we are going to design a new iterator: `MTData`. This iterator is built
on top of two TextReaders `src` and `tgt` that produce parallel sentences for source and
target languages.

In [4]:
mutable struct MTData
    src::TextReader        # reader for source language data
    tgt::TextReader        # reader for target language data
    batchsize::Int         # desired batch size
    maxlength::Int         # skip if source sentence above maxlength
    batchmajor::Bool       # batch dims (B,T) if batchmajor=false (default) or (T,B) if true.
    bucketwidth::Int       # batch sentences with length within bucketwidth of each other
    buckets::Vector        # sentences collected in separate arrays called buckets for each length range
    batchmaker::Function   # function that turns a bucket into a batch.
end

function MTData(src::TextReader, tgt::TextReader; batchmaker = arraybatch, batchsize = 128, maxlength = typemax(Int),
                batchmajor = false, bucketwidth = 10, numbuckets = min(128, maxlength ÷ bucketwidth))
    buckets = [ [] for i in 1:numbuckets ] # buckets[i] is an array of sentence pairs with similar length
    MTData(src, tgt, batchsize, maxlength, batchmajor, bucketwidth, buckets, batchmaker)
end

Base.IteratorSize(::Type{MTData}) = Base.SizeUnknown()
Base.IteratorEltype(::Type{MTData}) = Base.HasEltype()
Base.eltype(::Type{MTData}) = NTuple{2}

### iterate(::MTData)

Define the `iterate` function for the `MTData` iterator. `iterate` should return a
`(batch, state)` pair or `nothing` if there are no more batches.  The `batch` is a
`(x::Matrix{Int},y::Matrix{Int})` pair where `x` is a `(batchsize,srclength)` batch of
source language sentences and `y` is a `(batchsize,tgtlength)` batch of parallel target
language translations. The `state` is a pair of `(src_state,tgt_state)` which can be used
to iterate `d.src` and `d.tgt` to get more sentences.  `iterate(d)` without a second
argument should initialize `d` by emptying its buckets and calling `iterate` on the inner
iterators `d.src` and `d.tgt` without a state. Please review the documentation on
iterators from the last project.

To keep similar length sentences together `MTData` uses arrays of similar length sentence
pairs called buckets.  Specifically, the `(src_sentence,tgt_sentence)` pairs coming from
`src` and `tgt` are pushed into `d.buckets[i]` when the length of the source sentence is
in the range `((i-1)*d.bucketwidth+1):(i*d.bucketwidth)`. When one of the buckets reaches
`d.batchsize` `d.batchmaker` is called with the full bucket producing a 2-D batch, the
bucket is emptied and the batch is returned. If `src` and `tgt` are exhausted the
remaining partially full buckets are turned into batches and returned in any order. If the
source sentence length is larger than `length(d.buckets)*d.bucketwidth`, the last bucket
is used.

Sentences above a certain length can be skipped using the `d.maxlength` field, and
transposed `x,y` arrays can be produced using the `d.batchmajor` field.

In [5]:
function Base.iterate(d::MTData, state=nothing)
    # Your code here
    numbuckets=length(d.buckets)
    batchsize=d.batchsize
    if (state===nothing)
#         buckets=[ [] for i in 1:numbuckets ]
#         d=MTData(d.src, d.tgt, d.batchsize, d.maxlength, d.batchmajor, d.bucketwidth, buckets , d.batchmaker)
        d.buckets=[ [] for i in 1:numbuckets ]
        src_sentence, src_state=iterate(d.src)
        tgt_sentence, tgt_state=iterate(d.tgt)
#         println(src_sentence)
#         println(tgt_sentence)
    else
        src_state=state[1]
        tgt_state=state[2]
        x=iterate(d.src, src_state)
        y=iterate(d.tgt, tgt_state)
        if (x==nothing || y==nothing)
            for i in 1:numbuckets
                if length(d.buckets[i])>0
                    batch=d.batchmaker(d, d.buckets[i])
                    d.buckets[i]=[]
                    return (batch, (src_state, tgt_state))
                end
            end
            #print(d.buckets) this prints the buckets just before it ends we expect all empty buckets
            return nothing
        else
            src_sentence,src_state=x
            tgt_sentence,tgt_state=y
        end
    end
    
    while (length(src_sentence)>d.maxlength)
        #print("the src sentence is skipped")
        src_sentence, src_state=iterate(d.src, src_state)
        tgt_sentence, tgt_state=iterate(d.tgt, tgt_state)
    end
    
    
    for i in 1:numbuckets 
        if (length(src_sentence) > length(d.buckets)*d.bucketwidth)
            push!(d.buckets[numbuckets], (src_sentence, tgt_sentence))
            batch=d.batchmaker(d, d.buckets[numbuckets])
            d.buckets[numbuckets]=[]
            return (batch, (src_state, tgt_state))
        elseif (length(src_sentence) in ((i-1)*d.bucketwidth+1):(i*d.bucketwidth))
            push!(d.buckets[i], (src_sentence, tgt_sentence))
            if (length(d.buckets[i])==d.batchsize)
                batch=d.batchmaker(d, d.buckets[i])
                d.buckets[i]=[]
                return (batch, (src_state, tgt_state))
            else
                return iterate(d, (src_state, tgt_state))
            end
        end         
    end
end
#you still havent done batchmajor

### arraybatch

Define `arraybatch(d, bucket)` to be used as the default `d.batchmaker`. `arraybatch`
takes an `MTData` object and an array of sentence pairs `bucket` and returns a
`(x::Matrix{Int},y::Matrix{Int})` pair where `x` is a `(batchsize,srclength)` batch of
source language sentences and `y` is a `(batchsize,tgtlength)` batch of parallel target
language translations. Note that the sentences in the bucket do not have any `eos` tokens
and they may have different lengths. `arraybatch` should copy the source sentences into
`x` padding shorter ones on the left with `eos` tokens. It should copy the target
sentences into `y` with an `eos` token in the beginning and end of each sentence and
shorter sentences padded on the right with extra `eos` tokens.

In [6]:
function arraybatch(d::MTData, bucket)
    # Your code here
    longest_src=0
    longest_tgt=0
    batchsize=length(bucket)
#     x=[[] for i in 1:batchsize]
#     y=[[] for i in 1:batchsize]
    for sentence_pairs in bucket
        if (length(sentence_pairs[1])>longest_src)
            longest_src=length(sentence_pairs[1])
        end
        if (length(sentence_pairs[2])>longest_tgt)
            longest_tgt=length(sentence_pairs[2])
        end
    end
#     print(longest_src)
#     print(longest_tgt)
    x= Array{Int64,2}(undef, batchsize, longest_src)
    y= Array{Int64,2}(undef, batchsize, longest_tgt+2)
    for (i, sentence_pair) in enumerate(bucket)
        for k in 1:(longest_src-length(sentence_pair[1]))
            prepend!(sentence_pair[1], d.src.vocab.eos)
        end
        for (j, word) in enumerate(sentence_pair[1])
            x[i,j]=word
        end
#         print(x[i,:])
        prepend!(sentence_pair[2],d.tgt.vocab.eos)
        append!(sentence_pair[2], d.tgt.vocab.eos)
        for j in 1:(longest_tgt+2-length(sentence_pair[2]))
            append!(sentence_pair[2], d.tgt.vocab.eos)
        end
        for (j, word) in enumerate(sentence_pair[2])
            y[i,j]=word
        end
#         print(y[i,:])
    end
#     print(longest_src)
#     print(longest_tgt)
#     return (reshape(x, (batchsize, longest_src)), reshape(y, (batchsize, longest_tgt)))
#     print(size(x))
    return (x,y)
end

arraybatch (generic function with 1 method)

In [7]:
@info "Testing MTData"
dtrn = MTData(tr_train, en_train)
ddev = MTData(tr_dev, en_dev)
dtst = MTData(tr_test, en_test)

x,y = first(dtst)
@test length(collect(dtst)) == 48
@test size.((x,y)) == ((128,10),(128,24))
@test x[1,1] == tr_vocab.eos
@test x[1,end] != tr_vocab.eos
@test y[1,1] == en_vocab.eos
@test y[1,2] != en_vocab.eos
@test y[1,end] == en_vocab.eos

┌ Info: Testing MTData
└ @ Main In[7]:1


Test Passed

## Part 2. Sequence to sequence model without attention

In this part we will define a simple sequence to sequence encoder-decoder model for
machine translation.

In [8]:
mutable struct S2S_v1
    srcembed::Embed     # source language embedding
    encoder::RNN        # encoder RNN (can be bidirectional)
    tgtembed::Embed     # target language embedding
    decoder::RNN        # decoder RNN
    projection::Linear  # converts decoder output to vocab scores
    dropout::Real       # dropout probability to prevent overfitting
    srcvocab::Vocab     # source language vocabulary
    tgtvocab::Vocab     # target language vocabulary
end

### S2S_v1 constructor

Define the S2S_v1 constructor using your predefined layer types (Embed, Linear), and the
Knet RNN type. Please review the RNN documentation using `@doc RNN`, paying attention to
the following options in particular: `numLayers`, `bidirectional`, `dropout`, `dataType`,
`usegpu`. The last two are important if you experiment with array types other than the
default `KnetArray{Float32}`: make sure the RNNs use the same array type as the other
layers. Note that if the encoder is bidirectional, its `numLayers` should be half of the
decoder so that their hidden states match in size.

In [9]:
function S2S_v1(hidden::Int,         # hidden size for both the encoder and decoder RNN
                srcembsz::Int,       # embedding size for source language
                tgtembsz::Int,       # embedding size for target language
                srcvocab::Vocab,     # vocabulary for source language
                tgtvocab::Vocab;     # vocabulary for target language
                layers=1,            # number of layers
                bidirectional=false, # whether encoder RNN is bidirectional
                dropout=0)           # dropout probability
    # Your code here

    
    # embeddings
    vocab_size_source = length(srcvocab.w2i)
    vocab_size_target = length(tgtvocab.w2i)
    srcembed = Embed(vocab_size_source, srcembsz)
    tgtembed = Embed(vocab_size_target, tgtembsz)

    # encoder
    # [Ex, B, Tx] -> [Hx, Bx, Tx]
    # encoder.h will be [Hx, B]
    # decoder
    # [Ey, B, Ty] -> [Hy, B, Ty]
    # decoder.h will be [Hy, B]
    
    if bidirectional
        encoder = RNN(srcembsz, hidden; numLayers = layers,   dropout = dropout, bidirectional = true)
        decoder = RNN(tgtembsz, hidden; numLayers = (layers*2), dropout = dropout, bidirectional = false)
    else
        encoder = RNN(srcembsz, hidden; numLayers = layers, dropout = dropout, bidirectional = false)
        decoder = RNN(tgtembsz, hidden; numLayers = layers, dropout = dropout, bidirectional = false)
    end
    

    
    # projection
    # [hidden, B * Ty] -> [vocab_size_target, B * Ty]
    #projection = Linear(vocab_size_target, hidden)
    projection = Linear(hidden, vocab_size_target)
    
    # return model
    S2S_v1(srcembed, encoder, tgtembed, decoder, projection, dropout, srcvocab, tgtvocab)
    
end

S2S_v1

### S2S_v1 loss function

Define the S2S_v1 loss function that takes `src`, a source language minibatch, and `tgt`,
a target language minibatch and returns either a `(total_loss, num_words)` pair if
`average=false`, or `(total_loss/num_words)` average if `average=true`.

Assume that `src` and `tgt` are integer arrays of size `(B,Tx)` and `(B,Ty)` respectively,
where `B` is the batch size, `Tx` is the length of the longest source sequence, `Ty` is
the length of the longest target sequence. The `src` sequences only contain words, the
`tgt` sequences surround the words with `eos` tokens at the start and end. This allows
columns `tgt[:,1:end-1]` to be used as the decoder input and `tgt[:,2:end]` as the desired
decoder output.

Assume any shorter sentences in the batches have been padded with extra `eos` tokens on
the left for `src` and on the right for `tgt`. Don't worry about masking `src` for the
encoder, it doesn't have a significant effect on the loss. However do mask `tgt` before
`nll`: you do not want the padding tokens to be counted in the loss calculation.

Please review `@doc RNN`: in particular the `r.c` and `r.h` fields can be used to get/set
the cell and hidden arrays of an RNN (note that `0` and `nothing` act as special values).

RNNs take a dropout value at construction and apply dropout to the input of every layer if
it is non-zero. You need to handle dropout for other layers in the loss function or in
layer definitions as necessary.

In [10]:
function (s::S2S_v1)(src, tgt; average=true)
    
    # init encoder
    s.encoder.h = 0
    s.decoder.c = 0

    # ENCODER
    src_embed_out = s.srcembed(src)  
    #println("source embed out: ", summary(src_embed_out), size(src_embed_out))
    encoder_out = s.encoder(src_embed_out)
    #println("encoder_out:" , summary(encoder_out), size(encoder_out))

    # init decoder with encoder's h and c
    s.decoder.h = s.encoder.h
    s.decoder.c = s.encoder.c

    # DECODER
    tgt_embed_out = s.tgtembed(tgt[:,1:end-1])
    #println("target embed: ", summary(tgt_embed_out), size(tgt_embed_out))
    decoder_out = s.decoder(tgt_embed_out)
    #println("decoder_out: ", summary(decoder_out), size(decoder_out))
    
    
    # reshape
    decoder_out = reshape(decoder_out, (size(decoder_out)[1], size(decoder_out)[2] * size(decoder_out)[3]))
    #println("decoder_out (reshaped): ", summary(decoder_out), size(decoder_out))


    # Linear
    projection_out = s.projection(decoder_out)
    #println("projection_out: ", summary(projection_out), size(projection_out))

    # NLL
    scores = projection_out 
    #println("tgt: ", summary(tgt), size(tgt))

    answers = tgt[:,2:end]     
    answers = mask!(answers, s.tgtvocab.eos)    

    
    answers = reshape(answers, (1, size(answers)[1] * size(answers)[2]))
    #println("scores: ", summary(scores), size(scores))
    #println("answers: ", summary(answers), size(answers))

    nll(scores, answers; dims=1, average=average)
end

In [11]:
@info "Testing S2S_v1"
Knet.seed!(1)
model = S2S_v1(512, 512, 512, tr_vocab, en_vocab; layers=2, bidirectional=true, dropout=0.2)
(x,y) = first(dtst)
## Your loss can be slightly different due to different ordering of words in the vocabulary.
## The reference vocabulary starts with eos, unk, followed by words in decreasing frequency.
@test model(x,y; average=false) == (14097.471f0, 1432)

┌ Info: Testing S2S_v1
└ @ Main In[11]:1


Test Failed at In[11]:7
  Expression: model(x, y; average=false) == (14097.471f0, 1432)
   Evaluated: (14097.531f0, 1432) == (14097.471f0, 1432)


Test.FallbackTestSetException: There was an error during testing

### Loss for a whole dataset

Define a `loss(model, data)` which returns a `(Σloss, Nloss)` pair if `average=false` and
a `Σloss/Nloss` average if `average=true` for a whole dataset. Assume that `data` is an
iterator of `(x,y)` pairs such as `MTData` and `model(x,y;average)` is a model like
`S2S_v1` that computes loss on a single `(x,y)` pair.

In [12]:
function loss(model, data; average=true)
    if average; Knet.mean(model(x,y) for (x,y) in data)
    else
        total = 0
        counter = 0
        for (x,y) in data; out = model(x,y; average=average); total += out[1]; counter += out[2]; end
        return (total, counter)
    end
end

loss (generic function with 1 method)

In [13]:
@info "Testing loss"
@test loss(model, dtst, average=false) == (1.0429117f6, 105937)
# Your loss can be slightly different due to different ordering of words in the vocabulary.
# The reference vocabulary starts with eos, unk, followed by words in decreasing frequency.
# Also, because we do not mask src, different batch sizes may lead to slightly different
# losses. The test above gives (1.0429178f6, 105937) with batchsize==1.

┌ Info: Testing loss
└ @ Main In[13]:1


Test Failed at In[13]:2
  Expression: loss(model, dtst, average=false) == (1.0429117f6, 105937)
   Evaluated: (1.042915f6, 105937) == (1.0429117f6, 105937)


Test.FallbackTestSetException: There was an error during testing

### Training SGD_v1

The following function can be used to train our model. `trn` is the training data, `dev`
is used to determine the best model, `tst...` can be zero or more small test datasets for
loss reporting. It returns the model that does best on `dev`.

You should be able to get under 3.40 dev loss with the following settings in 10
epochs. The training speed on a V100 is about 3 mins/epoch or 40K words/sec, K80 is about
6 times slower. Using settings closer to the Luong paper (per-sentence loss rather than
per-word loss, SGD with lr=1, gclip=1 instead of Adam), you can get to 3.17 dev loss in
about 25 epochs. Using dropout and shuffling batches before each epoch significantly
improve the dev loss. You can play around with hyperparameters but I doubt results will
get much better without attention. To verify your training, here is the dev loss I
observed at the beginning of each epoch in one training session:
`[9.83, 4.60, 3.98, 3.69, 3.52, 3.41, 3.35, 3.32, 3.30, 3.31, 3.33]`

In [14]:
model = S2S_v1(512, 512, 512, tr_vocab, en_vocab; layers=2, bidirectional=true, dropout=0.2);

In [15]:
function load_params(model)
    encoder_w = Knet.load("encoder_w.jld2")["encoder_w"]
    decoder_w = Knet.load("decoder_w.jld2")["decoder_w"]
    proj = Knet.load("proj.jld2")["proj"]
    srcembed = Knet.load("srcmembed.jld2")["srcembed"]
    tgtembed = Knet.load("tgtembed.jld2")["tgtembed"]
    srcvocab = Knet.load("srcvocab.jld2")["srcvocab"]
    tgtvocab = Knet.load("tgtvocab.jld2")["tgtvocab"]
    dropout = Knet.load("dropout.jld2")["dropout"]
    model.encoder.w = encoder_w
    model.decoder.w = decoder_w
    model.projection = proj
    model.srcembed = srcembed
    model.tgtembed = tgtembed
    model.srcvocab = srcvocab
    model.tgtvocab = tgtvocab
    model.dropout = dropout
    model
end 

load_params (generic function with 1 method)

In [16]:
model = load_params(model);

In [ ]:
function train!(model, trn, dev, tst...)
    #println("trn: ", summary(trn))
    #println("dev: ", summary(dev))
    #println("tst: ", summary(tst), " content: ", summary(tst[1]))   
    bestmodel, bestloss = deepcopy(model), loss(model, dev)    
    progress!(adam(model, trn), steps=100) do y
        losses = [ loss(model, d) for d in (dev,tst...) ]
        if losses[1] < bestloss
            bestmodel, bestloss = deepcopy(model), losses[1]
        end
        return (losses...,)
    end
    #return model
    return bestmodel
end

In [ ]:
#epochs = 2
#ctrn = collect(dtrn)
#trnx10 = collect(flatten(shuffle!(ctrn) for i in 1:epochs))
#trn20 = ctrn[1:20]
#dev38 = collect(ddev);

In [ ]:
#model = train!(model, trnx10, dev38, trn20)

In [17]:
function save_model(model)
    model_name = "s2s_v1.jld2"
    Knet.save("encoder_w.jld2", "encoder_w", model.encoder.w)
    Knet.save("decoder_w.jld2", "decoder_w", model.decoder.w)
    Knet.save("proj.jld2", "proj", model.projection)
    Knet.save("srcmembed.jld2", "srcembed", model.srcembed)
    Knet.save("tgtembed.jld2", "tgtembed", model.tgtembed)
    Knet.save("srcvocab.jld2", "srcvocab", model.srcvocab)
    Knet.save("tgtvocab.jld2", "tgtvocab", model.tgtvocab)
    Knet.save("dropout.jld2", "dropout", model.dropout)
end

save_model (generic function with 1 method)

In [ ]:
#save_model(model)

### Generating translations

With a single argument, a `S2S_v1` object should take it as a batch of source sentences
and generate translations for them. After passing `src` through the encoder and copying
its hidden states to the decoder, the decoder is run starting with an initial input of all
`eos` tokens. Highest scoring tokens are appended to the output and used as input for the
subsequent decoder steps.  The decoder should stop generating when all sequences in the
batch have generated `eos` or when `stopfactor * size(src,2)` decoder steps are reached. A
correctly shaped target language batch should be returned.

In [23]:
function (s::S2S_v1)(src::Matrix{Int}; stopfactor = 3)
    
    #get source embeddings
    source_embed_out = s.srcembed(src)
    
    #init encoder
    s.encoder.h = 0
    s.encoder.c = 0

    #get encoder out
    encoder_out = s.encoder(source_embed_out)
    
    #set decoder to encoder out
    s.decoder.h = 0
    s.decoder.h = deepcopy(s.encoder.h)
    s.decoder.c = 0
    s.decoder.c = deepcopy(s.encoder.c)
    
    # get number of sentences in the batch
    batch_size = size(src)[1]
    
    # calculate stopping condition
    max_iters = stopfactor * size(src,2)
    
    # for each sentence, start with [eos]
    outputs = rand(s.tgtvocab.eos:s.tgtvocab.eos, (batch_size, 1))
        
    for i in 1:max_iters
    
        #println("iter ", i, " outputs: ", size(outputs))
        
        # get the embeddings of the current outputs, but only use the last timestep
        target_embed_out = s.tgtembed(outputs[:,i])
        #println("target embeddings:", size(target_embed_out))
        
        
        # decoder forward pass
        decoder_out = s.decoder(target_embed_out)
        #println("decoder output:", size(decoder_out))
        
        
        # projection forward pass
        proj_out = s.projection(decoder_out)
        
        # eliminate <unk>
        proj_out[s.tgtvocab.unk,1] = -10000

        #println("projection output: ", size(proj_out))
        
        #get best words
        #best_words should be (13,1)

        best_words = (x->x[1]).(argmax(proj_out; dims=1))
            
        outputs = hcat(outputs, transpose(best_words))
         
    end
    #println(size(s.decoder.h))
    
    outputs

    # DONT FORGET: for each sentence, only take the tokens up to eos.
    # you might also attempt to stop at eos but its gonna be harder
end

In [24]:
# Utility to convert int arrays to sentence strings
function int2str(y,vocab)
    y = vec(y)
    ysos = findnext(w->!isequal(w,vocab.eos), y, 1)
    ysos == nothing && return ""
    yeos = something(findnext(isequal(vocab.eos), y, ysos), 1+length(y))
    join(vocab.i2w[y[ysos:yeos-1]], " ")
end

int2str (generic function with 1 method)

In [45]:
@info "Generating some translations"
d = MTData(tr_dev, en_dev, batchsize=1) |> collect
(src,tgt) = rand(d)
out = model(src)
println("SRC: ", int2str(src,model.srcvocab))
println("REF: ", int2str(tgt,model.tgtvocab))
println("OUT: ", int2str(out,model.tgtvocab))
# Here is a sample output:
# SRC: çin'e 15 şubat 2006'da ulaştım .
# REF: i made it to china on february 15 , 2006 .
# OUT: i got to china , china , at the last 15 years .

┌ Info: Generating some translations
└ @ Main In[45]:1


SRC: şimdi bu konu bizi insanların ahlakı evrende yapmaya <unk> oldukları hareketlerin kaynağına götürüyor .
REF: now , this brings us to the sorts of moves that people are apt to make in the moral sphere .
OUT: so i 'm not sure that you can have a decent job for you . i 'm not going to be here .


### Calculating BLEU

BLEU is the most commonly used metric to measure translation quality. The following should
take a model and some data, generate translations and calculate BLEU.

In [36]:
function bleu(s2s,d::MTData)
    d = MTData(d.src,d.tgt,batchsize=32)
    reffile = d.tgt.file
    hypfile,hyp = mktemp()
    for (x,y) in progress(collect(d))
        g = s2s(x)
        for i in 1:size(y,1)
            println(hyp, int2str(g[i,:], d.tgt.vocab))
        end
    end
    close(hyp)
    isfile("multi-bleu.perl") || download("https://github.com/moses-smt/mosesdecoder/raw/master/scripts/generic/multi-bleu.perl", "multi-bleu.perl")
    run(pipeline(`cat $hypfile`,`perl multi-bleu.perl $reffile`))
    return hypfile
end

bleu (generic function with 1 method)

In [46]:
bleu(model, ddev)

┣████████████████████┫ [100.00%, 132/132, 00:53/00:53, 2.50i/s]  
perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
	LANGUAGE = (unset),
	LC_ALL = (unset),
	LC_CTYPE = "UTF-8",
	LANG = "en_US.UTF-8"
    are supported and installed on your system.
perl: warning: Falling back to the standard locale ("C").


BLEU = 7.40, 15.3/13.4/6.4/4.1 (BP=1.000, ratio=1.530, hyp_len=83756, ref_len=82502)


It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.


"/tmp/jl_KYRtcO"

Calculating dev BLEU takes about 45 secs on a V100. We get about 8.0 BLEU which is pretty
low. As can be seen from the sample translations a loss of ~3+ (perplexity ~20+) or a BLEU
of ~8 is not sufficient to generate meaningful translations.

To improve the quality of translations we can use more training data, different training
and model parameters, or preprocess the input/output: e.g. splitting Turkish words to make
suffixes look more like English function words may help. Other architectures,
e.g. attention and transformer, perform significantly better than this simple S2S model.

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*